In [ ]:
from helpers_given import *
from preprocess import *
from helpers import *
import seaborn as sns

Load data with helpers_given

In [ ]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("dataset/", sub_sample=False)

In [ ]:
nb_col = x_train.shape[1]
nb_rows = x_train.shape[0]

In [ ]:
y_01 = convert_minus1_to_0(y_train)
print("Number of people with MICHD: " + str(y_01[y_01==1].shape))
print("Number of people without MICHD: " + str(y_01[y_01==0].shape))

In [ ]:
with open('dataset/x_train.csv', 'r') as f:
    features_string = f.readline()
    features = features_string.split(',')
features = features[1:]

Preprocessing of run.py

In [ ]:
nb_nans = np.zeros(nb_col)
for i, col in enumerate(x_train.T):
    nb_nans[i] = np.count_nonzero(np.isnan(col))

In [ ]:
#nb_nans = number_of_NaNs(features, x_train)
reduced_data, reduced_features, Removed_features = removing_features(nb_nans, features, x_train, threshold=0.2)
#need to apply the same transfo to x_test
reduced_test = np.delete(x_test, Removed_features, 1)

In [ ]:
# Get feature correlation dictionary
feature_correlation_dict = create_dictionary_from_correlation(reduced_data,reduced_features,0.6)

max_corrr_feature_dict = {}

for key, val in feature_correlation_dict.items():
    max_corrr_feature_dict[key] = len(val)

# Sort the dictionary by value in descending order
max_corrr_feature_dict = {k: v for k, v in sorted(max_corrr_feature_dict.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
correlations = nan_corrcoef(reduced_data[:, 80:100])
# plt.figure(figsize=(20,20))
sns.heatmap(correlations, cmap="coolwarm")
plt.show()

In [ ]:
features_to_drop = []
for key in max_corrr_feature_dict.keys():
    features_to_drop.append(key)
    if len(features_to_drop) == 30:
        break

# Define features to keep
features_to_keep = []
for feature in reduced_features:
    if feature not in features_to_drop:
        features_to_keep.append(feature)

# Also replace some features with their calculated counterparts
origin_calculated_features = {
    'WEIGHT2' : 'WTKG3',
    'HEIGHT3' : 'HTM4',
    'ALCDAY5' : '_DRNKWEK',
    'FRUITJU1' : 'FTJUDA1_',
    'FRUIT1' : 'FRUTDA1_',
    'FVBEANS' : 'BEANDAY_',
    'FVGREEN' : 'GRENDAY_',
    'FVORANG' : 'ORNGDAY_',
    'VEGETAB1' : 'VEGEDA1_',
    'STRENGTH' : 'STRFREQ_'
}

# In features_to_keep replace the key of origin_calculated_features with the value
for key, val in origin_calculated_features.items():
    for i, feature in enumerate(features_to_keep):
        if key == feature:
            features_to_keep[i] = val

# Drop duplicates
features_to_keep = list(set(features_to_keep))

# Get the indices of the selected features
selected_features_indices = []
selected_features_indices_original = []
for feature in features_to_keep:
    selected_features_indices.append(reduced_features.index(feature))
    selected_features_indices_original.append(features.index(feature))

selected_features_indices = sorted(selected_features_indices)
selected_features_indices_original = sorted(selected_features_indices_original)



In [ ]:
# Create a new dataset with keeping the features that are in the selected_features_indices
reduced_data = reduced_data[:,selected_features_indices]
reduced_test = reduced_test[:,selected_features_indices]

# Also remove the features from the reduced_features list
reduced_features_2 = []
for feature in reduced_features:
    if feature in features_to_keep:
        reduced_features_2.append(feature)

# Remove redundant features
redundant_features = [ 'FMONTH','IDATE','IMONTH','IDAY','IYEAR', 'SEQNO', '_STATE', '_PSU', ]
# Get the indices of these features
redundant_features_indices = []
redundant_features_indices_original = []
for feature in redundant_features:
    if feature in reduced_features_2:
        redundant_features_indices.append(reduced_features_2.index(feature))
        redundant_features_indices_original.append(features.index(feature))

# Create a new dataset with removing the features that are in the selected_features_indices
reduced_data = np.delete(reduced_data, redundant_features_indices, 1)
reduced_features_2 = [reduced_features_2[i] for i in range(len(reduced_features_2)) if i not in redundant_features_indices]
reduced_test = np.delete(reduced_test, redundant_features_indices, 1)
selected_indices_original = [i for i in selected_features_indices_original if i not in redundant_features_indices_original]

# Replace nine values with NaNs
replace_nine_with_nan(reduced_data)
replace_nine_with_nan(reduced_test)

# Replace seven values with NaNs
replace_seven_with_nan(reduced_data)
replace_seven_with_nan(reduced_test)

# Replace 99 values with NaNs
replace_99_with_nan(reduced_data)
replace_99_with_nan(reduced_test)

# For the _DRNKWEK feature, replace 9990 with NaN
for i in range(reduced_data.shape[0]):
    if reduced_data[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        reduced_data[i, reduced_features_2.index('_DRNKWEK')] = np.nan
for i in range(reduced_test.shape[0]):
    if reduced_test[i, reduced_features_2.index('_DRNKWEK')] == 9990:
        reduced_test[i, reduced_features_2.index('_DRNKWEK')] = np.nan

In [ ]:
print(selected_indices_original)

In [ ]:
# Remove outliers
reduced_data = clean_outliers_modified(reduced_data)
reduced_test = clean_outliers_modified(reduced_test)

In [ ]:
# Replace NaNs with medians
reduced_median = replace_nans(reduced_data, method='median')
reduced_median_test = replace_nans(reduced_test, method='median')

# Standardize the data
std_x_med = standardize_data(reduced_median)
std_test_med = standardize_data(reduced_median_test)

In [ ]:
# plt.figure(figsize=(20,20))
sns.heatmap(np.corrcoef(reduced_median[:, 70:90].T), cmap="coolwarm")
plt.show()

#### Run the methods

In [ ]:
from implementations import *
NB_COL = std_x_med.shape[1] # corresponds to 'D' = number of features
NB_ROWS = std_x_med.shape[0] # corresponds to 'N' = number of observations/respondents
print("Number of features left: " + str(NB_COL))

In [ ]:
from cross_validation import *

In [ ]:
# CV on the two hyperparameters degree and lambda on regularized logistic regression adding polynomial terms
seed = 12
degrees = [1]
k_fold = 4
lambdas = np.linspace(0.1, 0.9, 33)
gamma = 0.5
max_iters = 50
initial_w = np.zeros(NB_COL)
k_indices = build_k_indices(y_01, k_fold, seed)
# for each degree, we compute the best lambdas and the associated rmse
best_lambdas = []
best_F1s = []
best_F1s_tr = []
best_lambdas_tr = []
plot_F1_te = np.zeros((len(degrees),len(lambdas)))
# vary degree
for degree in degrees:
    # cross validation
    F1_te = []
    F1_tr = []
    for lambda_ in lambdas:
        F1_te_tmp = []
        F1_tr_tmp = []
        for k in range(k_fold):
            F1_training, F1_test = cross_validation(y_01, std_x_med,initial_w, max_iters, gamma, k_indices, k, lambda_, degree)
            F1_te_tmp.append(F1_test)
            F1_tr_tmp.append(F1_training)
        F1_te.append(np.mean(F1_te_tmp))
        F1_tr.append(np.mean(F1_tr_tmp))
        plot_F1_te[degrees.index(degree),np.where(lambdas==lambda_)[0][0]] = np.mean(F1_te_tmp)
    print(f" Finished for degree: {degree}")
    ind_lambda_opt = np.argmax(F1_te)
    ind_lambda_opt_tr = np.argmax(F1_tr)
    best_lambdas.append(lambdas[ind_lambda_opt])
    best_lambdas_tr.append(lambdas[ind_lambda_opt_tr])
    best_F1s.append(F1_te[ind_lambda_opt])
    best_F1s_tr.append(F1_tr[ind_lambda_opt_tr])
ind_best =  np.nanargmax(best_F1s)      
best_degree = degrees[ind_best]
best_lambda = best_lambdas[ind_best]
best_F1 = best_F1s[ind_best]

In [ ]:
print(best_degree)
print(best_lambda)
print(best_F1)

### Plotting evolution of training and test F1-score as a function of lambda or degree

In [ ]:
plt.plot(lambdas,F1_te)
plt.xlabel("lambda")
plt.ylabel("F1-score")
plt.title("Evolution of F1-score as a function of lambda")
plt.grid(True)
plt.savefig("F1-score_lambda")

In [ ]:
plt.figure(figsize=(10,10))
for i in range(len(degrees)):
    plt.subplot(2,2,i+1)
    plt.plot(lambdas,plot_F1_te[i,:])
    plt.xlabel('Lambda')
    plt.ylabel('F1-score')
    plt.title(f'F1-score as a function of lambda with polynomial degree = {degrees[i]}')
    plt.grid(True)
plt.show()

### Run regularized logistic regression with best param

In [ ]:
tx = build_poly(std_x_med, best_degree)

In [ ]:
w_reg, loss_reg = reg_logistic_regression(y_01, tx, initial_w = np.zeros(tx.shape[1]), max_iters = 100, gamma = 0.5, lambda_ = best_lambda)
print("(logistic) loss: " + str(loss_reg))
y_reg = convert_0_to_minus1(convert_predict(tx @ w_reg))
print(y_reg)
# accuracy
p_reg = compute_accuracy(y_train, y_reg) #percentage of false predictions
print("accuracy: " + str(p_reg))

In [ ]:
tx_test = build_poly(std_test_med, best_degree)
y_pred = convert_0_to_minus1(convert_predict(tx_test @ w_reg))

In [ ]:
create_csv_submission(test_ids, y_pred, "best_result_1")

### F1-score and accuracy of all methods without hyperparameter finetuning

In [ ]:
# GD
w_mse_gd, loss_mse_gd = mean_squared_error_gd(y_01, std_x_med, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
y_mse_gd = convert_predict(std_x_med @ w_mse_gd)
print(f"The F1-score is: {compute_f1(y_01,y_mse_gd)} and the accuracy is : {compute_accuracy(y_01,y_mse_gd)}")

In [ ]:
# SGD
w_mse_sgd, loss_mse_sgd = mean_squared_error_sgd(y_01, std_x_med, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
y_mse_sgd = convert_predict(std_x_med @ w_mse_sgd)
print(f"The F1-score is: {compute_f1(y_01,y_mse_sgd)} and the accuracy is : {compute_accuracy(y_01,y_mse_sgd)}")

In [ ]:
# Least squares
w_ls, loss_ls = least_squares(y_01, std_x_med)
y_ls = convert_predict(std_x_med @ w_ls)
print(f"The F1-score is: {compute_f1(y_01,y_ls)} and the accuracy is : {compute_accuracy(y_01,y_ls)}")

In [ ]:
# Ridge regression
w_rr, loss_rr = ridge_regression(y_01, std_x_med, lambda_ = 0.5)
y_rr = convert_predict(std_x_med @ w_rr)
print(f"The F1-score is: {compute_f1(y_01,y_rr)} and the accuracy is : {compute_accuracy(y_01,y_rr)}")

In [ ]:
# logistic regression (GD)
w_lrgd, loss_lrgd = logistic_regression(y_01, std_x_med, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001)
y_lrgd = convert_predict(std_x_med @ w_lrgd)
print(f"The F1-score is: {compute_f1(y_01,y_lrgd)} and the accuracy is : {compute_accuracy(y_01,y_lrgd)}")
#log sgd
#w_lrsgd, loss_lrsgd = logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)

In [ ]:
w_reg, loss_reg = reg_logistic_regression(y_01, std_x_med, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.001, lambda_ = 0.5)
y_reg = convert_predict(std_x_med @ w_reg)
print(f"The F1-score is: {compute_f1(y_01,y_reg)} and the accuracy is : {compute_accuracy(y_01,y_reg)}")
# reg log sgd
#w_reg_lrsgd, loss_reg_lrsgd = reg_logistic_regression_SGD(y_train, x_train, initial_w = np.zeros(NB_COL), max_iters = 50, gamma = 0.1, lambda_ = 0.5)